# Sequential feature selection algorithm - Class of Greedy serach algorithm

An alternative way to reduce the complexity of the model and avoid overfitting is dimensionality reduction via feature selection, which is especially useful for unregularized models.

Two main categories of dimensionality  reduction techniques,
- feature selection : we select a subset of original feature
- feature extraction : we derive infromation from feature set to construct a new feature subspace.

In sequential feaature slection algorithm, we reduce initial d-dimensional feature space to a k-dimensional feature subspace where k<d. 
Reason - automatically selectionn a subset of features which are more relevent to the problem; and imporve computational efficiency or noise, which can be useful for lagorithm that dont support regularization.

**Classic sequential feature selection algorithm is - Sequential backward selection**


In [1]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
class SBS():
    def __init__(self, estimator, k_features, scoring = accuracy_score, 
                 test_size = 0.25, random_state = 1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state
        
    def fit(self, X, y):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = self.test_size,
                                                            random_state = self.random_state)
        
        dim = X_train.shape[1]
        self.indices_  = tuple(range(dim))
        self.subsets_  = [self.indices_]
        score = self._calc_score(X_train, y_train, X_test, y_test, self.indices_)
        
        self.scores_ = [score]
        
        while dim > self.k_features:
            scores = []
            subsets = []
            
            for p in combinations(self.indices_, r = dim -1):
                score = self._calc_score(X_train, y_train, X_test, y_test, p)
                
                scores.append(score)
                subsets.append(p)
                
            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1
            
            self.scores_.append(scores[best])
            
        self.k_score_ = self.scores_[-1]
        
        return self
    
    def transform(self, X):
        return X[:, self.indices_]
    
    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

In [3]:
import numpy as np
import pandas as pd

df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/''machine-learning-databases/wine/wine.data', header = None)

df_wine.columns = ['ClassLabel', 'Alcohol', 'MalicAcid', 'Ash', 'AlcalinityOfAsh', 'Magnesium', 'TotalPhenols', 'Flavanoids', 
                  'NonflavanoidsPhenols', 'Proanthocyanins', 'ColorIntensity', 'Hue', 'OD280/OD315OfDilutedWines', 'Proline']

print(df_wine.head(5))
print(df_wine.shape)

   ClassLabel  Alcohol  MalicAcid   Ash  AlcalinityOfAsh  Magnesium  \
0           1    14.23       1.71  2.43             15.6        127   
1           1    13.20       1.78  2.14             11.2        100   
2           1    13.16       2.36  2.67             18.6        101   
3           1    14.37       1.95  2.50             16.8        113   
4           1    13.24       2.59  2.87             21.0        118   

   TotalPhenols  Flavanoids  NonflavanoidsPhenols  Proanthocyanins  \
0          2.80        3.06                  0.28             2.29   
1          2.65        2.76                  0.26             1.28   
2          2.80        3.24                  0.30             2.81   
3          3.85        3.49                  0.24             2.18   
4          2.80        2.69                  0.39             1.82   

   ColorIntensity   Hue  OD280/OD315OfDilutedWines  Proline  
0            5.64  1.04                       3.92     1065  
1            4.38  1.05     

In [4]:
from sklearn.model_selection import train_test_split

#X, y = df_wine.iloc[:,1:], df_wine.iloc[:,0] # not in array form
X, y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(178, 13) (178,) (124, 13) (124,) (54, 13) (54,)


In [6]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_std = ss.fit_transform(X_train)
X_test_std = ss.fit_transform(X_test)

In [7]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)

sbs = SBS(knn, k_features = 1)
sbs.fit(X_train_std, y_train)
